In [1]:
# Importando os módulos que serão utilizados

import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import time
import dill

# Configurando o acesso do tensorflow aos processadores do computador
# no caso, está sendo selecionada a primeira placa de vídeo listada

gpus = tf.config.experimental.list_physical_devices('GPU') # Listando as placas de vídeo
tf.config.experimental.set_memory_growth(gpus[0], True)    # Selecionando a primeira GPU e configurando

# Importando os módulos contendo as funções criadas no projeto

from aux_func_V3 import *
import API_Numpy
import API_TensorFlow

In [2]:
Δx=0.05
CFL=0.5

In [3]:
Sim, Sim_step, DerivadaEspacial, Get_weights=create_simulation(API_Numpy,burgers_equation,WENO_Z)
WENO_Z_ref=lambda u0, Δx:Sim(u0,Δx*CFL, Δx, CFL, FronteiraPeriodica)

In [4]:
with open('datasets\\Dados temporais.bkp','rb') as file:
    y,dy=dill.load(file)

In [5]:
indice = np.arange(y.shape[0])
np.random.shuffle(indice)
data_x = y.astype('float64')[indice]
data_y = dy.astype('float64')[indice]

In [6]:
# Conjunto de dados de treino
train_x = data_x#[:-20000]
train_y = data_y#[:-20000]

# Conjunto de dados de validação
test_x = data_x#[-20000:]
test_y = data_y#[-20000:]

In [7]:
# Criando uma camada de Burgers que integra o WENO à rede neural
Sim_layer = WENO_temporal(Δx, CFL, Δx*CFL, FronteiraPeriodica)

In [8]:
# Definindo o input da rede e o otimizador de treino
input_x   = keras.layers.Input([train_x.shape[1]], dtype='float64')
optimizer = keras.optimizers.Adam(learning_rate=10**-3, beta_1=0.9, beta_2=0.999, clipnorm=1.0)

# Criando a rede neural
Network = keras.Model(input_x, Sim_layer(input_x))
# Configurando a função de perda e o otimizador
Network.compile(loss=MES_OF(), optimizer=optimizer, metrics=['mean_absolute_error'])
# Carregando os pesos da rede neural treinados
#Network.load_weights('Modelo artigo')

(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)


In [9]:
# Treinando a rede neural
Network.fit(
    train_x                           , # Dados de treino
    train_y                           , # Dados de treino
    validation_data = (test_x, test_y), # Dados de validação
    batch_size      = 1024            , # Tamanho do batch
    epochs          = 100             , # Número de epochs
    steps_per_epoch = 300             , # Número de batchs por epoch
    shuffle         = True              # Aleatorização dos batchs
)

# Batch: pacote de dados utilizados antes de uma atualização dos pesos da rede
# Epoch: rodada de treino da rede neural, em geral percorre todo o conjunto de dados

Epoch 1/100
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
300/300 [==============================] - ETA: 0s - loss: 1.0240e-04 - mean_absolute_error: 5.4030e-04(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
300/300 [==============================] - 586s 2s/step - loss: 1.0240e-04 - mean_absolute_error: 5.4030e-04 - val_loss: 1.0213e-04 - val_mean_absolute_error: 5.4030e-04
Epoch 2/100
  4/300 [..............................] - ETA: 9:23 - loss: 9.9949e-05 - mean_absolute_error: 5.3855e-04

KeyboardInterrupt: 

In [10]:
# Salvando os pesos treinados
Network.save_weights('Modelos treinados\\Modelo Rede temporal - 1')

In [11]:
# Carregando os pesos treinados
Network.load_weights('Modelos treinados\\Modelo Rede temporal - 1')

In [12]:
# Definindo o input da rede e o otimizador de treino
input_x   = keras.layers.Input([200], dtype='float64')
optimizer = keras.optimizers.Adam(learning_rate=10**-3, beta_1=0.9, beta_2=0.999, clipnorm=1.0)

# Criando a rede neural
Network = keras.Model(input_x, Sim_layer(input_x))
# Configurando a função de perda e o otimizador
Network.compile(loss='MSE', optimizer=optimizer, metrics=['mean_absolute_error'])

(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)


In [13]:
# Calculando os erros de previsão utilizando o WENO-Z em uma malha mais fina 
# como solução de referência e depois calculando o WENO-Z e o WENO-Z com a 
# modificação da rede neural numa malha mais grossa
Δx_ref = 0.01
Δx = Δx_ref                                # Distância espacial dos pontos na malha mais grossa utilizada
x  = tf.range(-1, 1, Δx, dtype=float_pres) # Gerando a malha de pontos no espaço unidimensional
                             # Distância espacial dos pontos na malha utilizada

# Condição inicial do artigo do WENO-Z
#-------------------------------------------------------------------------------------------
# Função definida no artigo
f_test = lambda x: -(-tf.math.sin(np.pi*x) - 0.5 * x**3 + \
    tf.where(x < 0, tf.constant(0.0, dtype=float_pres), tf.constant(1.0, dtype=float_pres)))

full_U=tf.expand_dims(f_test(x),axis=0)

net_u   = Network(full_U)

(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)


In [14]:
# Calculando os erros de previsão utilizando o WENO-Z em uma malha mais fina 
# como solução de referência e depois calculando o WENO-Z e o WENO-Z com a 
# modificação da rede neural numa malha mais grossa

k=1

Δx = Δx_ref/k                              # Distância espacial dos pontos na malha mais grossa utilizada
x  = tf.range(-1, 1, Δx, dtype=float_pres) # Gerando a malha de pontos no espaço unidimensional
                             # Distância espacial dos pontos na malha utilizada
# Condição inicial do artigo do WENO-Z

full_U=tf.expand_dims(f_test(x),axis=0)

debug_u   = WENO_Z_ref(full_U,Δx)         # Previsão com o WENO-Z modificado pela rede neural

(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)


In [15]:
# Calculando os erros de previsão utilizando o WENO-Z em uma malha mais fina 
# como solução de referência e depois calculando o WENO-Z e o WENO-Z com a 
# modificação da rede neural numa malha mais grossa

k=2

Δx = Δx_ref/k                               # Distância espacial dos pontos na malha mais grossa utilizada
x  = tf.range(-1, 1, Δx, dtype=float_pres) # Gerando a malha de pontos no espaço unidimensional
                             # Distância espacial dos pontos na malha utilizada

full_U=tf.expand_dims(f_test(x),axis=0)

ref_full   = tf.gather(WENO_Z_ref(full_U,Δx),np.arange(net_u.shape[-1])*k,axis=-1)             # Previsão com o WENO-Z modificado pela rede neural

# Armazenando ambos os erros de previsão
error = tf.stack([net_u-ref_full,debug_u-ref_full],axis=0)
error = tf.squeeze(error)

(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)


In [16]:
# Calculando média dos erros de acordo com a norma L2, L1 ou L-inf

# Norma L2:
#--------------------------------------------------------------------------------------
desv_error = tf.math.reduce_sum(error**2, axis=-1)**0.5
#--------------------------------------------------------------------------------------
print('L2:')
# Erro médio para a rede neural e o WENO-Z (respectivamente)
print(desv_error.numpy())
# Diferença entre os erros (WENO-Z - rede neural)
# print((desv_error[1]-desv_error[0]).numpy())
# Razão entre os erros (rede neural/WENO-Z)
print((desv_error[0]/desv_error[1]).numpy())
print('\n')

# Norma L1:
#--------------------------------------------------------------------------------------
desv_error = tf.math.reduce_sum(tf.abs(error), axis=-1)
#--------------------------------------------------------------------------------------
print('L1:')
# Erro médio para a rede neural e o WENO-Z (respectivamente)
print(desv_error.numpy())
# Diferença entre os erros (WENO-Z - rede neural)
# print((desv_error[1]-desv_error[0]).numpy())
# Razão entre os erros (rede neural/WENO-Z)
print((desv_error[0]/desv_error[1]).numpy())
print('\n')

# Noma L-inf:
#-------------------------------------------------------------------------------------
desv_error = tf.math.reduce_max(tf.abs(error), axis=-1)
#--------------------------------------------------------------------------------------
print('L-inf:')
# Erro médio para a rede neural e o WENO-Z (respectivamente)
print(desv_error.numpy())
# Diferença entre os erros (WENO-Z - rede neural)
# print((desv_error[1]-desv_error[0]).numpy())
# Razão entre os erros (rede neural/WENO-Z)
print((desv_error[0]/desv_error[1]).numpy())

L2:
[0.2778602  0.27500402]
1.010385958155875


L1:
[0.70187221 0.69486957]
1.010077631954841


L-inf:
[0.27491981 0.27217183]
1.0100964671514348


In [17]:
Sim, Sim_step, DerivadaEspacial, Get_weights=create_simulation(API_Numpy,burgers_equation,WENO_Z)
WENO_Z_ref=lambda u0, Δx:DerivadaEspacial(u0,Δx, FronteiraPeriodica)

In [18]:
Δx_ref = 0.01

def f(x):
    return np.sin(np.pi*x)

def df(x):
    return np.pi*np.cos(np.pi*x)


In [19]:
# Definindo o input da rede e o otimizador de treino
input_x   = keras.layers.Input([200], dtype='float64')
optimizer = keras.optimizers.Adam(learning_rate=10**-3, beta_1=0.9, beta_2=0.999, clipnorm=1.0)

# Criando a rede neural
Network = keras.Model(input_x, Sim_layer(input_x))
# Configurando a função de perda e o otimizador
Network.compile(loss='MSE', optimizer=optimizer, metrics=['mean_absolute_error'])

(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)
(None, 201, None)
(None, 201, 3)


In [20]:
Δx1 = Δx_ref
x1  = np.arange(-1, 1, Δx1)
x1  = np.expand_dims(x1,0)
u1  = f(x1)
du1 = df(x1)
y1=Network(u1, Δx1)

print(u1.shape)

(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 201, 3)
(1, 200)


In [21]:
# Definindo o input da rede e o otimizador de treino
input_x   = keras.layers.Input([400], dtype='float64')
optimizer = keras.optimizers.Adam(learning_rate=10**-3, beta_1=0.9, beta_2=0.999, clipnorm=1.0)

# Criando a rede neural
Network = keras.Model(input_x, Sim_layer(input_x))
# Configurando a função de perda e o otimizador
Network.compile(loss='MSE', optimizer=optimizer, metrics=['mean_absolute_error'])

(None, 401, None)
(None, 401, 3)
(None, 401, None)
(None, 401, 3)
(None, 401, None)
(None, 401, 3)
(None, 401, None)
(None, 401, 3)
(None, 401, None)
(None, 401, 3)
(None, 401, None)
(None, 401, 3)


In [22]:
Δx2 = Δx_ref/2
x2  = np.arange(-1, 1, Δx2)
x2  = np.expand_dims(x2,0)
u2  = f(x2)
du2 = df(x2)
y2=Network(u2, Δx2)

print(u2.shape)

(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 401, 3)
(1, 400)


In [23]:
# Gerando os gráficos a partir de funções do matplotlib

print(np.sum(abs(y1 - du1))/np.sum(abs(y2 - du2)))

# print(np.mean(abs(WENO_Z_ref(u1, Δx1) - df(x1 + Δx1/2)))/np.mean(abs(WENO_Z_ref(u2, Δx2) - df(x2 + Δx2/2))))

# fig = plt.figure(1, constrained_layout=True,figsize=(6,6))
# ax  = fig.add_subplot(1,1,1);
# # ax.set_ylim(-2, 2);
# # ax.set_xlim(0,1);
# line = ax.plot(x1, WENO_Z_ref(u1, Δx1) - du1)
# # line = ax.plot(x1, du1)
# line = ax.plot(x2, WENO_Z_ref(u2, Δx2) - du2)
# # line = ax.plot(x2, du2)
# hfig = display(fig, display_id=True)


0.49998893595719707
